In [1]:
#coding:utf8
import os
from torch.utils import data
import numpy as np
import xml.etree.ElementTree as ET
import torch as t
from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
import sys
sys.path.append('C:/Users/SEONGGYUN/ATAE-LSTM/ATAE-LSTM')
import Ipynb_importer
from config import opt
from data.Embedding import Emb

importing Jupyter notebook from C:/Users/SEONGGYUN/ATAE-LSTM/ATAE-LSTM\data\Embedding.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SEONGGYUN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<center>
                string word =dict=> int index =t.nn.Embedding=> t.Tensor<br>
</center>

In [3]:
class AspClas(data.Dataset):
    
    def __init__(self, root=opt.train_data_root, train=True, test=False, emb=None):
      
        # temporary list to save string format data
        # free after init
        self.raw_cases = []
        # list to save tensor format data
        # used when __getitem__() is called
        self.cases = []
        
        # dictionary used to transform polarity
        self.polar = {'positive':[2], 'neutral':[1], 'negative':[0]}
        
        # load root = 'restaurants-trial.xml'
        xml = ET.parse(root)
        for s in tqdm(xml.findall('sentence')):
            if s.find('aspectTerms'):
                text = s.find('text').text
                asps = s.find('aspectTerms').findall('aspectTerm')
                for asp in asps: 
                    if asp.attrib['polarity'] in self.polar:
                        self.raw_cases.append((text, asp.attrib['term'], asp.attrib['polarity']))
        
        # division
        if test:
            pass
        elif train:
            self.raw_cases = self.raw_cases[:int(0.7*len(self.raw_cases))]
        else:
            self.raw_cases = self.raw_cases[int(0.7*len(self.raw_cases)):]
        
        """
        # shuffle
        np.random.seed(100)
        self.raw_cases = np.random.permutation(self.raw_cases)
        """
        # dataloader already has this function
        
        
        if emb is None:
            self.emb = Emb()
        else:
            self.emb = emb
        
        # transform
        self._addall2emb_()
        self.transform()
        
        return
    
    def __getitem__(self, index):
        '''
        sentence-term-polarity
        '''
        return self.cases[index]
    
    def __len__(self):
        return len(self.cases)
    
    def _addall2emb_(self):
        for (raw_text, raw_term, raw_polarity) in self.raw_cases:
            self.emb._add_word_(raw_text)
            self.emb._add_word_(raw_term)
        return
    
    def transform(self):
        '''
        transform the strings into word index
        transform the polar into one-hot of classes
        '''
        
        # refresh self.cases
        self.cases = []
        
        # transform
        for (raw_text, raw_term, raw_polarity) in self.raw_cases:
            # transform text and term using emb.tokenize
            
            # text
            text_tensor = self.emb.tokenize(raw_text, max_length=opt.max_seq_len)
            
            # term
            term_tensor = self.emb.tokenize(raw_term, max_length=opt.max_terms_len)
            
            # polarity
            polarity_tensor = t.Tensor(self.polar[raw_polarity]).long()
            
            self.cases.append(
                (
                    text_tensor,
                    term_tensor,
                    polarity_tensor
                )
            )
            
        return

In [4]:
if __name__=='__main__':
    testDataset = AspClas(opt.test_data_root)
    testLoader = DataLoader(
        testDataset,
        batch_size = 2,
        shuffle = True
    )
    print(testDataset[0])
    print(testDataset[1])
    print(list(testLoader)[0])

100%|██████████| 100000/100000 [00:06<00:00, 16539.26it/s]


Embedding : successfully input 100000 pretrained word embeddings while 0 failed
(tensor([ 2,  3,  4,  5,  6,  7,  8,  9,  3, 10, 11, 12, 13,  5,  3, 14, 11, 15,
        16, 16, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0]), tensor([4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0]), tensor([2]))
(tensor([ 2,  3,  4,  5,  6,  7,  8,  9,  3, 10, 11, 12, 13,  5,  3, 14, 11, 15,
        16, 16, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  